In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('copenhagen_data.csv')

In [7]:
df.head()

,municipality,municipality code,year,gender,age,ethnicity,education,income,income observations,weighted_inc_mun,days in hospital,days in hospital observations,weighted_hosp_days_mun,number of hospitalizations,number of hospitalizations observations,weighted_hospitalizations_mun
0,Denmark,0,2005,Men,18-29,Danish,Higher education,NaN,NaN,NaN,0.186,30425.0,0.001379,0.049,30425.0,0.000364
1,Denmark,0,2005,Men,18-29,Danish,No higher education,NaN,NaN,NaN,0.269,296267.0,0.019420,0.076,296267.0,0.005493
2,Denmark,0,2005,Men,18-29,Immigrants and descendants,Higher education,NaN,NaN,NaN,0.159,1925.0,0.000075,0.047,1925.0,0.000022
3,Denmark,0,2005,Men,18-29,Immigrants and descendants,No higher education,NaN,NaN,NaN,0.256,26048.0,0.001625,0.078,26048.0,0.000496
4,Denmark,0,2005,Men,18-29,Non-western immigrants and descendants,Higher education,NaN,NaN,NaN,0.155,1096.0,0.000041,0.047,1096.0,0.000013


In [17]:
# Perform one-hot encoding on the 'gender' column
one_hot_encoded_gender = pd.get_dummies(df.gender, prefix='gender')

# Concatenate the one-hot encoded columns to the original DataFrame
df_gender = pd.concat([df.drop('gender', axis=1), one_hot_encoded_gender], axis=1)
df_gender


,municipality,municipality code,year,age,ethnicity,education,income,income observations,weighted_inc_mun,days in hospital,days in hospital observations,weighted_hosp_days_mun,number of hospitalizations,number of hospitalizations observations,weighted_hospitalizations_mun,gender_Men,gender_Women
0,Denmark,0,2005,18-29,Danish,Higher education,NaN,NaN,NaN,0.186,30425.0,0.001379,0.049,30425.0,0.000364,1,0
1,Denmark,0,2005,18-29,Danish,No higher education,NaN,NaN,NaN,0.269,296267.0,0.019420,0.076,296267.0,0.005493,1,0
2,Denmark,0,2005,18-29,Immigrants and descendants,Higher education,NaN,NaN,NaN,0.159,1925.0,0.000075,0.047,1925.0,0.000022,1,0
3,Denmark,0,2005,18-29,Immigrants and descendants,No higher education,NaN,NaN,NaN,0.256,26048.0,0.001625,0.078,26048.0,0.000496,1,0
4,Denmark,0,2005,18-29,Non-western immigrants and descendants,Higher education,NaN,NaN,NaN,0.155,1096.0,0.000041,0.047,1096.0,0.000013,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39600,Greater Copenhagen,1,2020,60-65,Danish,No higher education,327995.938342,19143.0,9789.862554,NaN,NaN,NaN,NaN,NaN,NaN,0,1
39601,Greater Copenhagen,1,2020,60-65,Immigrants and descendants,Higher education,360832.894995,1075.0,604.801301,NaN,NaN,NaN,NaN,NaN,NaN,0,1
39602,Greater Copenhagen,1,2020,60-65,Immigrants and descendants,No higher education,233379.566602,2257.0,821.282403,NaN,NaN,NaN,NaN,NaN,NaN,0,1
39603,Greater Copenhagen,1,2020,60-65,Non-western immigrants and descendants,Higher education,258666.804848,369.0,148.821334,NaN,NaN,NaN,NaN,NaN,NaN,0,1


In [19]:
women_dk = df[(df.gender == 'Women') & (df.municipality == 'Denmark')]
women_cph = df[(df.gender == 'Women') & (df.municipality == 'Greater Copenhagen')]

In [20]:
women_dk 

,municipality,municipality code,year,gender,age,ethnicity,education,income,income observations,weighted_inc_mun,days in hospital,days in hospital observations,weighted_hosp_days_mun,number of hospitalizations,number of hospitalizations observations,weighted_hospitalizations_mun
32,Denmark,0,2005,Women,18-29,Danish,Higher education,NaN,NaN,NaN,0.357,43149.0,0.003754,0.112,43149.0,0.001179
33,Denmark,0,2005,Women,18-29,Danish,No higher education,NaN,NaN,NaN,0.413,272456.0,0.027420,0.129,272456.0,0.008575
34,Denmark,0,2005,Women,18-29,Immigrants and descendants,Higher education,NaN,NaN,NaN,0.258,3207.0,0.000202,0.096,3207.0,0.000075
35,Denmark,0,2005,Women,18-29,Immigrants and descendants,No higher education,NaN,NaN,NaN,0.431,27150.0,0.002851,0.150,27150.0,0.000994
36,Denmark,0,2005,Women,18-29,Non-western immigrants and descendants,Higher education,NaN,NaN,NaN,0.327,1713.0,0.000136,0.109,1713.0,0.000046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19500,Denmark,0,2020,Women,60-65,Danish,No higher education,310519.203,117529.0,14874.976629,NaN,NaN,NaN,NaN,NaN,NaN
19501,Denmark,0,2020,Women,60-65,Immigrants and descendants,Higher education,347248.702,3460.0,489.710615,NaN,NaN,NaN,NaN,NaN,NaN
19502,Denmark,0,2020,Women,60-65,Immigrants and descendants,No higher education,238199.117,6903.0,670.194422,NaN,NaN,NaN,NaN,NaN,NaN
19503,Denmark,0,2020,Women,60-65,Non-western immigrants and descendants,Higher education,285827.752,1491.0,173.702003,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
women_cph

,municipality,municipality code,year,gender,age,ethnicity,education,income,income observations,weighted_inc_mun,days in hospital,days in hospital observations,weighted_hosp_days_mun,number of hospitalizations,number of hospitalizations observations,weighted_hospitalizations_mun
17721,Greater Copenhagen,1,2005,Women,18-29,Danish,Higher education,NaN,NaN,NaN,0.273738,16965.0,0.004705,0.096133,16965.0,0.001652
17722,Greater Copenhagen,1,2005,Women,18-29,Danish,No higher education,NaN,NaN,NaN,0.355233,67801.0,0.024400,0.122644,67801.0,0.008425
17723,Greater Copenhagen,1,2005,Women,18-29,Immigrants and descendants,Higher education,NaN,NaN,NaN,0.212001,1413.0,0.000303,0.082674,1413.0,0.000118
17724,Greater Copenhagen,1,2005,Women,18-29,Immigrants and descendants,No higher education,NaN,NaN,NaN,0.396178,12382.0,0.004970,0.140789,12382.0,0.001766
17725,Greater Copenhagen,1,2005,Women,18-29,Non-western immigrants and descendants,Higher education,NaN,NaN,NaN,0.205432,555.0,0.000116,0.088315,555.0,0.000050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39600,Greater Copenhagen,1,2020,Women,60-65,Danish,No higher education,327995.938342,19143.0,9789.862554,NaN,NaN,NaN,NaN,NaN,NaN
39601,Greater Copenhagen,1,2020,Women,60-65,Immigrants and descendants,Higher education,360832.894995,1075.0,604.801301,NaN,NaN,NaN,NaN,NaN,NaN
39602,Greater Copenhagen,1,2020,Women,60-65,Immigrants and descendants,No higher education,233379.566602,2257.0,821.282403,NaN,NaN,NaN,NaN,NaN,NaN
39603,Greater Copenhagen,1,2020,Women,60-65,Non-western immigrants and descendants,Higher education,258666.804848,369.0,148.821334,NaN,NaN,NaN,NaN,NaN,NaN
